In [ ]:
%pip install requests beautifulsoup4 pandas urllib3   

In [1]:
# Import required libraries
import requests
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET
import json
import pandas as pd
import os
import urllib3
from requests.adapters import HTTPAdapter
from urllib3.util.ssl_ import create_urllib3_context

In [2]:
# Import urllib3 if not already importe
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Create a custom adapter to handle SSL
class CustomAdapter(HTTPAdapter):
	def init_poolmanager(self, *args, **kwargs):
		context = create_urllib3_context()
		context.options |= 0x4  # Enable legacy renegotiation
		context.check_hostname = False
		kwargs['ssl_context'] = context
		return super(CustomAdapter, self).init_poolmanager(*args, **kwargs)

# Create a session with the custom adapter
session = requests.Session()
session.verify = False
session.mount('https://', CustomAdapter())

# ERLDC

In [3]:
# Define the URL for the ERLDC API
erldc_api_url = "https://app.erldc.in/api/LiveDataScheduler/Get/RegionStatistics"

# Send a GET request to the URL
response_erldc = session.get(erldc_api_url)

# Parse the response as JSON
data_erldc = response_erldc.json()
data_erldc

{'StatId': 439167,
 'Date': '050325',
 'Time': '131134',
 'Freq': '49.92',
 'RevNo': '81',
 'DemandMet': '21657.85156',
 'DSMMet': '593.084351'}

In [4]:
# Convert the dictionary to a DataFrame by passing it as a list
df_erldc = pd.DataFrame([data_erldc])
# Alternatively, you could transpose the data to get keys as columns
# df_erldc = pd.DataFrame.from_dict(data_erldc, orient='index').T
df_erldc

,StatId,Date,Time,Freq,RevNo,DemandMet,DSMMet
0,439167,050325,131134,49.92,81,21657.85156,593.084351


# SRLDC

In [5]:
# Define the URL f or the JSON API endpoint
srldc_api_url = "https://www.srldc.in/indexPageDataInEvery5min"

# Make a GET request to the JSON API endpoint using the existing session
response_srldc = session.get(srldc_api_url)
print(response_srldc)

<Response [200]>


In [6]:
# Import pprint
from pprint import pprint

# Parse the response as JSON
data_srldc = response_srldc.json()
# Print the response
pprint(data_srldc)

{'allIndiaDemand': 200217.11,
 'dsmRate': 454.07,
 'dsmRateForIndex': 200217,
 'frequency': 50.4,
 'localDate': {'chronology': {'calendarType': 'iso8601', 'id': 'ISO'},
               'dayOfMonth': 5,
               'dayOfWeek': 'WEDNESDAY',
               'dayOfYear': 64,
               'era': 'CE',
               'leapYear': False,
               'month': 'MARCH',
               'monthValue': 3,
               'year': 2025},
 'localDateForUpdate': 1741160940000,
 'revisionNo': None,
 'sRDemand': 66788.78,
 'sRDemandForIndex': 66788}


# NRLDC

In [7]:
# Define the URL for the target website
nrldc_url = "https://newnr.nrldc.in/"

# Make a GET request to the target website
response_nrldc = session.get(nrldc_url)

In [8]:
soup_nrldc = BeautifulSoup(response_nrldc.text, 'html.parser')

content_divs = soup_nrldc.find_all('div', class_='content')

for content_div in content_divs:
    h2 = content_div.find('h2', class_='m-0')
    p = content_div.find('p')
    if h2 and p:
        print(f"{h2.text}: {p.text}")

Frequency: 50.44 Hz
All India Demand: 200735 MW
NR Demand: 42914 MW
Peak Demand Today: 58285 MW
NR Generation: 57873 MW
NR RE Generation: 26252 MW
Current Sch Revision: 100
Last Updated On: 05-03-25 13:15:00


# WRLDC

In [9]:
# Define the URL for the target website
wrldc_url = "https://wrldc.in/content/English/index.aspx"

# Make a GET request to the target website
wrldc_response = session.get(wrldc_url, verify=False)
wrldc_response.status_code

200

In [10]:
# Parse the response using BeautifulSoup
wrldc_soup = BeautifulSoup(wrldc_response.text, 'html.parser')

box_elements = wrldc_soup.find_all('div', class_='box')

# Create a dictionary to store the results
data_dict = {}

# Loop through each box element
for box in box_elements:
    # Find the span and strong elements
    span_text = box.find('span').text.strip()
    strong_element = box.find('strong')
    
    # Get the ID and value
    if strong_element:
        data_id = strong_element.get('id')
        data_value = strong_element.text.strip()
        data_dict[data_id] = {
            'label': span_text,
            'value': data_value
        }

# Convert to DataFrame for better visualization
import pandas as pd
df = pd.DataFrame.from_dict(data_dict, orient='index')
print(df)

                                      label              value
dataDateTime                      date time  05 Mar 2025 13:17
dataDemand                           Demand              67917
dataFrequency                     Frequency              50.45
dataDeviationRate  Deviation Rate (Rs/Unit)                  0
dataRenewable                     Renewable              23536
dataRevision                   Revision nos                100


In [11]:
df

,label,value
dataDateTime,date time,05 Mar 2025 13:17
dataDemand,Demand,67917
dataFrequency,Frequency,50.45
dataDeviationRate,Deviation Rate (Rs/Unit),0
dataRenewable,Renewable,23536
dataRevision,Revision nos,100


# Google sheet setup

In [13]:
!uv pip install gspread oauth2client pandas

Resolved 23 packages in 613ms
Prepared 12 packages in 487ms
         If the cache and target directories are on different filesystems, hardlinking may not be supported.
         If this is intentional, set `export UV_LINK_MODE=copy` or use `--link-mode=copy` to suppress this warning.
Installed 12 packages in 579ms
 + cachetools==5.5.2
 + google-auth==2.38.0
 + google-auth-oauthlib==1.2.1
 + gspread==6.2.0
 + httplib2==0.22.0
 + oauth2client==4.1.3
 + oauthlib==3.2.2
 + pyasn1==0.6.1
 + pyasn1-modules==0.4.1
 + pyparsing==3.2.1
 + requests-oauthlib==2.0.0
 + rsa==4.9


In [19]:
!uv pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib gspread

Audited 4 packages in 66ms


In [ ]:
from oauth2client.service_account import ServiceAccountCredentials
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.ssl_ import create_urllib3_context
import urllib3

# Disable SSL warnings
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Custom Adapter to handle SSL
class CustomAdapter(HTTPAdapter):
    def init_poolmanager(self, *args, **kwargs):
        context = create_urllib3_context()
        context.options |= 0x4  # Enable legacy renegotiation
        context.check_hostname = False
        kwargs['ssl_context'] = context
        return super(CustomAdapter, self).init_poolmanager(*args, **kwargs)

# Create a session with the custom adapter
session = requests.Session()
session.verify = False
session.mount('https://', CustomAdapter())

# Fetch API Data
srldc_api_url = "https://www.srldc.in/indexPageDataInEvery5min"
response = session.get(srldc_api_url)
data = response.json()

In [ ]:
type(response.text)
response.text

str

In [42]:
response_json = json.dumps(response.json())
print(response_json)

{"localDate": {"year": 2025, "month": "MARCH", "chronology": {"id": "ISO", "calendarType": "iso8601"}, "era": "CE", "leapYear": false, "dayOfMonth": 5, "monthValue": 3, "dayOfWeek": "WEDNESDAY", "dayOfYear": 64}, "localDateForUpdate": 1741163220000, "dsmRate": 454.07, "sRDemand": 66082.8, "allIndiaDemand": 198882.64, "frequency": 50.0, "revisionNo": null, "sRDemandForIndex": 66082, "dsmRateForIndex": 198882}


In [43]:
type(response_json)

str

In [40]:
pprint(data)

{'allIndiaDemand': 198882.64,
 'dsmRate': 454.07,
 'dsmRateForIndex': 198882,
 'frequency': 50.0,
 'localDate': {'chronology': {'calendarType': 'iso8601', 'id': 'ISO'},
               'dayOfMonth': 5,
               'dayOfWeek': 'WEDNESDAY',
               'dayOfYear': 64,
               'era': 'CE',
               'leapYear': False,
               'month': 'MARCH',
               'monthValue': 3,
               'year': 2025},
 'localDateForUpdate': 1741163220000,
 'revisionNo': None,
 'sRDemand': 66082.8,
 'sRDemandForIndex': 66082}


In [16]:
# Remove unwanted fields
data.pop('localDate', None)
data.pop('localDateForUpdate', None)

pprint(data)

{'allIndiaDemand': 200118.92,
 'dsmRate': 454.07,
 'dsmRateForIndex': 200118,
 'frequency': 50.38,
 'revisionNo': None,
 'sRDemand': 66746.0,
 'sRDemandForIndex': 66746}


In [36]:
type(data)

dict

In [37]:
data.json()
type(data)

AttributeError: 'dict' object has no attribute 'json'

In [29]:
import gspread
from google.oauth2.service_account import Credentials

# Define the scope
scopes = [
    'https://www.googleapis.com/auth/spreadsheets',
]

# Define the credentials
credentials = Credentials.from_service_account_file(
    'credentials.json',
    scopes=scopes
)

# Create a client
client = gspread.authorize(credentials)

# Open the Google Sheet
sheet_id = "1VVpzD2BzF4FMqxVglStMc-6rO51onkdGT-hOLiJqv24"

workbook = client.open_by_key(sheet_id)

In [30]:
sheets = workbook.worksheets()
sheets

[<Worksheet 'Sheet1' id:0>, <Worksheet 'Sheet2' id:1831781442>]

In [31]:
sheet1 = workbook.worksheet('Sheet1')
sheet1

<Worksheet 'Sheet1' id:0>

In [33]:
sheet2 = workbook.worksheet('Sheet2')
sheet2

<Worksheet 'Sheet2' id:1831781442>

In [34]:
# Change the sheet1 by the output of the API
sheet1.update([list(data.keys()), list(data.values())])
print("Sheet1 updated successfully!")

Sheet1 updated successfully!
